In [21]:
import sys
import requests
from time import sleep

from tqdm import tqdm
import pymongo
from pymongo import MongoClient
from bson.objectid import ObjectId
import gensim

In [22]:
sys.path.append('../')

In [3]:
from src import settings

In [4]:
client = MongoClient(settings.MONGODB_SETTINGS['host'])
db = client[settings.MONGODB_SETTINGS['db']]
col = db[settings.MONGODB_SETTINGS['collection']]

In [5]:
from src.models import make_texts_corpus
from src.utils import editorJs_data_to_text
import json
from gensim.matutils import cossim
from gensim import similarities

In [6]:
id2word = gensim.corpora.Dictionary.load(settings.PATH_DICTIONARY)
corpus = gensim.corpora.MmCorpus('../models/corpus.mm')
lda_model = gensim.models.ldamodel.LdaModel.load('../models/LDA.model')

INFO : loading Dictionary object from ../models/id2word.dictionary
INFO : Dictionary lifecycle event {'fname': '../models/id2word.dictionary', 'datetime': '2022-12-30T16:55:29.939429', 'gensim': '4.2.0', 'python': '3.9.6 (tags/v3.9.6:db3ff76, Jun 28 2021, 15:26:21) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19045-SP0', 'event': 'loaded'}
INFO : loaded corpus index from ../models/corpus.mm.index
INFO : initializing cython corpus reader from ../models/corpus.mm
INFO : accepted corpus with 177 documents, 4087 features, 36185 non-zero entries
INFO : loading LdaModel object from ../models/LDA.model
INFO : loading expElogbeta from ../models/LDA.model.expElogbeta.npy with mmap=None
INFO : setting ignored attribute state to None
INFO : setting ignored attribute dispatcher to None
INFO : setting ignored attribute id2word to None
INFO : LdaModel lifecycle event {'fname': '../models/LDA.model', 'datetime': '2022-12-30T16:55:29.963434', 'gensim': '4.2.0', 'python': '3.9.6 (tags/v3.

In [7]:
index = similarities.MatrixSimilarity(lda_model[corpus])

INFO : creating matrix with 177 documents and 57 features


In [8]:
for post in col.find():
    print(post['title'], 'idrs: ', post['idrs'] if 'idrs' in post else False)

ĐỒ ÁN THIẾT KẾ WEBSITE BLOG SITE idrs:  0
Regular Expression trong Javascript idrs:  2
Các bài tập java cơ bản idrs:  3
Những câu hỏi lập trình khi pv (2): bài toán ATM idrs:  4
Ví dụ sắp xếp nổi bọt với Java idrs:  5
Chương trình lọc thư rác bằng Java, code ví dụ idrs:  6
Đọc ghi file Excel trong Java sử dụng Apache POI idrs:  7
THREAD AND THREAD POOLS IN JAVA idrs:  8
Sử dụng CountDownLatch trong Java idrs:  9
Multithreading: Java Synchronized Blocks idrs:  10
Understand synchronized and improve lock performance in Java idrs:  11
Front-End Web Developers idrs:  12
Material Design for Web Design idrs:  13
Những kỹ năng cần phải có của một Web Designer idrs:  14
Rest API - cách Frontend và Backend nói chuyện với nhau qua ví dụ cụ thể idrs:  15
Điều cần chú ý để xây dựng RESTful API idrs:  16
Dựng một API RESTful đơn giản sử dụng Express và MongoDB idrs:  17
xây dựng API với Napa idrs:  18
Trí tuệ nhân tạo với Swift idrs:  19
Lợi ích và nguy cơ của trí tuệ nhân tạo (AI) idrs:  20
Liệu c

In [9]:
# main_post = col.find_one({"_id": ObjectId("6388c04363cdee37de74f09c")})
main_post = col.find_one({"idrs": 165})
main_post['title']

'Văn hóa Tây Bắc - những mạch nguồn chảy mãi'

In [10]:
content = editorJs_data_to_text(json.loads(main_post["content"]))
text_corpus = make_texts_corpus([content])
bow = id2word.doc2bow(next(text_corpus))

In [11]:
vector_doc = lda_model[bow]

In [12]:
sims = index[vector_doc]

In [13]:
sims = sorted(enumerate(sims), key=lambda item: -item[1])
#for doc_position, doc_score in sims:
#    print(doc_score, doc_position)

In [20]:
print(main_post['title'])
i=0
for idrs, distance in sims:
    article = col.find_one({"idrs": idrs})
    print(i, idrs, distance, article['title'])
    i+=1
    if(i==100): break
    

Văn hóa Tây Bắc - những mạch nguồn chảy mãi
0 165 1.0 Văn hóa Tây Bắc - những mạch nguồn chảy mãi
1 166 0.9341245 Sắc màu văn hóa các dân tộc vùng Tây Bắc
2 110 0.87812924 Chuẩn bị đi du lịch? Cẩm nang từ A-Z giúp bạn có kỳ nghỉ TRỌN VẸN
3 163 0.84791946 Áo dài - nét đẹp đặc sắc của văn hoá Việt Nam
4 161 0.80711055 Chiếc áo dài – Nét đẹp văn hóa truyền thống của người phụ nữ Việt Nam
5 162 0.7901239 Lịch sử phát triển áo dài Việt Nam qua các thời kỳ
6 107 0.75350416 Những phong tục trong ngày Tết cổ truyền
7 29 0.73424983 Chia sẻ kinh nghiệm du lịch bụi đơn giản nhưng rất cần thiết cho du khách
8 164 0.7148472 Ý nghĩa tà áo dài truyền thống Việt Nam – giá trị qua các thời kỳ lịch sử
9 109 0.70836735 Giữ gìn, phát huy nét đẹp văn hóa Tết cổ truyền của Dân tộc Việt Nam
10 108 0.5590424 Phong tục truyền thống trong ngày Tết cổ truyền của người Việt
11 30 0.4939037 Kinh nghiệm du lịch Hạ Long tự túc cho người lần đầu mới đi
12 105 0.42101556 Phong tục Tết cổ truyền Việt Nam
13 112 0.37909

In [15]:
arr = [tup[0] for tup in sims[:10]]
if 'idrs' in main_post: 
    if main_post['idrs'] in arr: arr.remove(main_post['idrs'])
print(main_post['idrs'])
list(arr)

165


[166, 110, 163, 161, 162, 107, 29, 164, 109]